In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.compose import make_column_transformer, ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.metrics import mean_absolute_percentage_error
from math import sqrt

data = pd.read_csv('./ds-mobilidade-urbana-fortaleza/df_input.csv', sep=';')

day_of_week_translator = {
    0: "Domingo",
    1: "Segunda",
    2: "Terca",
    3: "Quarta",
    4: "Quinta",
    5: "Sexta",
    6: "Sabado",
}

# busline_filter = '41'
# data_model = data[data.linha.isin([busline_filter])]
data_model = data
data_model

In [ ]:
plt.figure(figsize=(15,5))
plt.xlabel('Tempo')
plt.ylabel('Quantidade')
plt.plot(data['hora'], data['validations_per_hour'], 'b--')
plt.show()
plt.clf()
plt.figure(figsize=(15,5))
plt.plot(data['d_semana'], data['validations_per_hour'], 'b--')
plt.show()
plt.clf()
plt.figure(figsize=(15,5))
plt.plot(data['d_ano'], data['validations_per_hour'], 'b--')
plt.show()

In [ ]:
time_steps = 30  #TAMANHO DA JANELA
test_size = 120  #HORIZONTE DE PREVISÃO
one_hot_encoder = OneHotEncoder(sparse=False)

data_model[['domingo','segunda', 'terca', 'quarta', 'quinta', 'sexta', 'sabado']] = one_hot_encoder.fit_transform(data_model['d_semana'].values.reshape(-1,1))
data_model[['marco', 'abril', 'maio', 'junho', 'julho', 'agosto', 'setembro', 'outubro', 'novembro', 'dezembro', 'janeiro']] = one_hot_encoder.fit_transform(data_model['mes'].values.reshape(-1,1))

train_size = int(len(data_model)-(test_size))
train, test = data_model.iloc[0:train_size], data_model.iloc[(train_size-time_steps):len(data_model)]
print(len(train), len(test))

def create_dataset(X, y, time_steps=1):
    Xs, ys = [], []
    for i in range(len(X) - time_steps):
        v = X.iloc[i:(i + time_steps),0].to_numpy()
        v = np.append(v,X.iloc[i + time_steps,0])#linha
        v = np.append(v,X.iloc[i + time_steps,3])#d_semana
        # v = np.append(v,X.iloc[i + time_steps,10])#d_semanaonehotencoded -> provou ser PIOR
        # v = np.append(v,X.iloc[i + time_steps,11])#d_semanaonehotencoded -> provou ser PIOR
        # v = np.append(v,X.iloc[i + time_steps,12])#d_semanaonehotencoded -> provou ser PIOR
        # v = np.append(v,X.iloc[i + time_steps,13])#d_semanaonehotencoded -> provou ser PIOR
        # v = np.append(v,X.iloc[i + time_steps,14])#d_semanaonehotencoded -> provou ser PIOR
        # v = np.append(v,X.iloc[i + time_steps,15])#d_semanaonehotencoded -> provou ser PIOR
        # v = np.append(v,X.iloc[i + time_steps,16])#d_semanaonehotencoded -> provou ser PIOR
        v = np.append(v,X.iloc[i + time_steps,4])#hr_sin
        v = np.append(v,X.iloc[i + time_steps,5])#hr_cos
        v = np.append(v,X.iloc[i + time_steps,7])#d_mes -> provou ser melhor com a adicao deste componente
        # v = np.append(v,X.iloc[i + time_steps,8])#d_ano
        Xs.append(v)
        ys.append(y.iloc[i + time_steps])
    return np.array(Xs), np.array(ys)

X_train, y_train = create_dataset(train, train['validations_per_hour'], time_steps)
X_test, y_test = create_dataset(test, test['validations_per_hour'], time_steps)


In [ ]:
X_train

In [ ]:
X_test

In [ ]:
model = RandomForestRegressor(n_jobs=10).fit(X_train,y_train)
#model = LinearRegression.fit(X_train,y_train)
preds = []

base_teste = np.copy(X_test)

for i in range(len(base_teste)):
    y_pred = model.predict(np.array([base_teste[i]]))[0]

    preds.append(y_pred)
    
    for k in range(len(preds)):
        
        if i<len(base_teste):
            if k < time_steps:
                if(i<len(base_teste)-1):
                    base_teste[i+1][(time_steps-1)-k] = preds[(len(preds)-1)-k]

print(len(preds))

dados_real = data_model.iloc[(train_size):len(data_model),2].to_numpy()

len(dados_real)

df_real_predito = pd.DataFrame({'real':dados_real,'predito':preds})

print(df_real_predito)
# X_train[0]

plt.figure(figsize=(15,5))
# plt.plot(range(len(y_train)),y_train, 'g--')
plt.plot(range(len(df_real_predito['predito'])),df_real_predito['predito'], 'g--')
plt.plot(range(len(df_real_predito['real'])),df_real_predito['real'], 'b')
# plt.xlim(0,30)
print('rmse=',sqrt(mean_squared_error(df_real_predito['real'].array,df_real_predito['predito'].array)))
print('mae=',mean_absolute_error(df_real_predito['real'].array,df_real_predito['predito'].array))
print('mape=',mean_absolute_percentage_error(df_real_predito['real'].array,df_real_predito['predito'].array))
print('r2=',r2_score(df_real_predito['real'].array,df_real_predito['predito'].array))

plt.show()